In [38]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
model_name = "richielo/small-e-czech-finetuned-ner-wikiann"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)
nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = """
„BIS je připravena poskytovat prezidentu republiky maximální informační podporu, tak jak je to běžné ve vyspělých demokratických státech, a to všemi formami včetně pravidelných bezpečnostních briefingů,“ citovala BIS Koudelku.

Pavlův přístup ostře kontrastuje s přístupem dosluhují hlavy státu Miloše Zemana. Ten opakovaně kritizoval BIS i samotného Koudelku, jehož odmítal jmenovat generálem.

Návrhy mu předložila vláda Andreje Babiše (ANO) i Petra Fialy (ODS). Další návrh na povýšení současná vláda již dříve avizovala na období po volbách, až se funkce ujme nový prezident. Pavel před volbami řekl, že by Koudelku do generálské hodnosti jmenoval.
"""

In [39]:
def filter_person_entities(entities: list):
    return [ent for ent in entities if 'PER' in ent['entity']]

def get_person_entities(text: str, ner_pipeline):
    entities = ner_pipeline(text)
    return filter_person_entities(entities)

def replacer(s, newstring, index, nofail=False):
    if not nofail and index not in range(len(s)):
        raise ValueError("index outside given string")

    # if not erroring, but the index is still not in the correct range..
    if index < 0:  # add it to the beginning
        return newstring + s
    if index > len(s):  # add it to the end
        return s + newstring

    # insert the new string between "slices" of the original
    return s[:index] + newstring + s[index + 1:]

def get_masked_text(text: str, ner_pipeline):
    masked_text = text
    entities = get_person_entities(example, ner_pipeline)
    for ent in entities:
        running_index = ent.get('start')
        while True:            
            masked_text = replacer(masked_text, '*', running_index)
            running_index += 1
            if running_index == ent.get('end'):
                break
    return masked_text

print('---Original---')
print(example)
print('---Masked---')
print(get_masked_text(example, nlp))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


---Original---

„BIS je připravena poskytovat prezidentu republiky maximální informační podporu, tak jak je to běžné ve vyspělých demokratických státech, a to všemi formami včetně pravidelných bezpečnostních briefingů,“ citovala BIS Koudelku.

Pavlův přístup ostře kontrastuje s přístupem dosluhují hlavy státu Miloše Zemana. Ten opakovaně kritizoval BIS i samotného Koudelku, jehož odmítal jmenovat generálem.

Návrhy mu předložila vláda Andreje Babiše (ANO) i Petra Fialy (ODS). Další návrh na povýšení současná vláda již dříve avizovala na období po volbách, až se funkce ujme nový prezident. Pavel před volbami řekl, že by Koudelku do generálské hodnosti jmenoval.

---Masked---

„BIS je připravena poskytovat prezidentu republiky maximální informační podporu, tak jak je to běžné ve vyspělých demokratických státech, a to všemi formami včetně pravidelných bezpečnostních briefingů,“ citovala BIS Koudelku.

Pavlův přístup ostře kontrastuje s přístupem dosluhují hlavy státu ****** ******. Ten op

In [40]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

class NER_masker_cz:
    
    def __init__(self, model_name="richielo/small-e-czech-finetuned-ner-wikiann"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForTokenClassification.from_pretrained(model_name)
        self.NER = pipeline("ner", model=model, tokenizer=tokenizer)
        
    def _filter_entities(self, entities: list, entity_filter: list) -> list:
        return [ent for ent in entities for ent_type in entity_filter if ent_type in ent['entity']]

    def _get_filtered_entities(self, text: str, entity_filter: list) -> list:
        entities = self.NER(text)
        return self._filter_entities(entities, entity_filter)

    def _replacer(self, s, newstring, index, nofail=False):
        if not nofail and index not in range(len(s)):
            raise ValueError("index outside given string")

        # if not erroring, but the index is still not in the correct range..
        if index < 0:  # add it to the beginning
            return newstring + s
        if index > len(s):  # add it to the end
            return s + newstring

        # insert the new string between "slices" of the original
        return s[:index] + newstring + s[index + 1:]

    def get_masked_text(self, text: str, entity_filter: list = ['B-PER', 'I-PER']):
        masked_text = text
        entities = self._get_filtered_entities(example, entity_filter)
        for ent in entities:
            running_index = ent.get('start')
            while True:            
                masked_text = replacer(masked_text, '*', running_index)
                running_index += 1
                if running_index == ent.get('end'):
                    break
        return masked_text
        

In [41]:
masker = NER_masker_cz()
print(masker.get_masked_text(example))


„BIS je připravena poskytovat prezidentu republiky maximální informační podporu, tak jak je to běžné ve vyspělých demokratických státech, a to všemi formami včetně pravidelných bezpečnostních briefingů,“ citovala BIS Koudelku.

Pavlův přístup ostře kontrastuje s přístupem dosluhují hlavy státu ****** ******. Ten opakovaně kritizoval BIS i samotného ********, jehož odmítal jmenovat generálem.

Návrhy mu předložila vláda ******* ****** (ANO) i ***** ***** (ODS). Další návrh na povýšení současná vláda již dříve avizovala na období po volbách, až se funkce ujme nový prezident. ***** před volbami řekl, že by ***delku do generálské hodnosti jmenoval.

